# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1O9aE_Hen5smf_a6TsbVF6DM0ms4Ykj6s?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [50]:
#!pip install selenium

In [51]:
# Passo 1: Entrar na Internet (abrir o navegador)
from selenium import webdriver
navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')

In [52]:
# Passo 2: Importar a base de dados
import pandas as pd

tabela = pd.read_excel('commodities.xlsx')
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [53]:
# .send_keys("meu nome é leticia") -> escrever nele
# .click() -> clicar nele
# .get_attribute() -> pegar uma informação dele
# import unicodedata
# produto = unicodedata.normalize("NFKD", produto).encode("ascii", "ignore")

for linha in tabela.index:
    produto = tabela.loc[linha, 'Produto']
    
    # entrar no site melhor cambio: https://www.melhorcambio.com/milho-hoje
    link = f'https://www.melhorcambio.com/{produto}-hoje'
    link = link.replace('ó', 'o').replace('ã', 'a').replace('á', 'a').replace('ç', 'c').replace('ú', 'u').replace('é', 'e')
    print(link)
    navegador.get(link)

    # pegar a cotação do milho
    cotacao = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    cotacao = cotacao.replace('.', '').replace(',', '.')
    print(cotacao)

    # na coluna Preço Atual do Milho, preecher a cotação do milho
    tabela.loc[linha, 'Preço Atual'] = float(cotacao)
    

print('Acabou')
display(tabela)

https://www.melhorcambio.com/Milho-hoje
56.51
https://www.melhorcambio.com/Soja-hoje
132.56
https://www.melhorcambio.com/Boi-hoje
267.30
https://www.melhorcambio.com/Petroleo-hoje
377.18
https://www.melhorcambio.com/Algodao-hoje
395.63
https://www.melhorcambio.com/Acucar-hoje
148.83
https://www.melhorcambio.com/Cafe-hoje
1043.39
https://www.melhorcambio.com/Ouro-hoje
313.05
https://www.melhorcambio.com/Tilapia-hoje
9.64
Acabou


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,NaN
1,Soja,163.59,132.56,NaN
2,Boi,282.20,267.30,NaN
3,Petróleo,424.37,377.18,NaN
4,Algodão,497.76,395.63,NaN
5,Açúcar,136.23,148.83,NaN
6,Café,1092.87,1043.39,NaN
7,Ouro,321.77,313.05,NaN
8,Tilápia,9.05,9.64,NaN


In [54]:
# Passo 6: Decidir quais produtos a gente vai comprar
display(tabela)

tabela['Comprar'] = tabela['Preço Atual'] < tabela['Preço Ideal']
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,NaN
1,Soja,163.59,132.56,NaN
2,Boi,282.20,267.30,NaN
3,Petróleo,424.37,377.18,NaN
4,Algodão,497.76,395.63,NaN
5,Açúcar,136.23,148.83,NaN
6,Café,1092.87,1043.39,NaN
7,Ouro,321.77,313.05,NaN
8,Tilápia,9.05,9.64,NaN


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,True
1,Soja,163.59,132.56,True
2,Boi,282.20,267.30,True
3,Petróleo,424.37,377.18,True
4,Algodão,497.76,395.63,True
5,Açúcar,136.23,148.83,False
6,Café,1092.87,1043.39,True
7,Ouro,321.77,313.05,True
8,Tilápia,9.05,9.64,False


In [55]:
# fechar o navegador
navegador.quit()

# Passo 7: Exportar a base de dados atualizados
tabela.to_excel('commodities_atualizado.xlsx', index=False)